In [19]:
%pip install pygame
%pip install scikit-sound

     |################################| 11.4MB 11.8MB/s 
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
%conda install ffplay

Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - ffplay

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/linux-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import glob
from IPython.display import Audio
import logging
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
import time
import logging
from IPython.display import Audio

from IDRnD.utils import *
from IDRnD.augmentations import *
from IDRnD.dataset import *
from IDRnD.resnet import resnet50
from IDRnD.focalloss import FocalLoss
import numpy as np

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from sklearn.model_selection import StratifiedKFold, train_test_split

%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log",
                    filemode="w+")

In [2]:
X, y = get_train_data()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [3]:
train_transform = transforms.Compose([
    #RandomParameter(VTLP, [[0.8, 1.2]], p=0.5),
    #MinMaxChunkScaler(),
    #Normalize(),
    #RandomParameter(RandomNoise, [[0.01, 0.1]]),
    #RandomParameter(Shift, [[2000, 32000]]),
    #RandomParameter(TimeStretch, [[0.75, 1.3]]),
    #RandomParameter(PitchShift, [[-8, 8]]),
    #RandomParameter(Distortion, [[-1, -0.3], [.3, 1.]]),
    ToMellSpec(n_mels=128),
    #GetMFCC(),
    #PadOrClip(320),
    #Normalize(),
    #ToTensor(),
    #transforms.ToTensor(),
])

In [4]:
train_dataset = SimpleMelDataset(X, y, "../data/files/raw_mels/", train_transform)
#val_dataset = Base_Dataset(X_val, y_val, train_transform)

In [6]:
ls /data_common_voices/ru/

clips/       dev.tsv  invalidated.tsv  ru.tar.gz  train.tsv
clips_wavs/  foo.wav  other.tsv        test.tsv   validated.tsv


In [31]:
base_path = "/data_common_voices/en/clips_wavs/"
common_voice_files = os.listdir(base_path)
common = [os.path.join(base_path, file) for file in common_voice_files]
common_dataset = BaseDataset(common, np.ones_like(common, dtype=np.int16), train_transform)

In [29]:
for file in common_voice_files:
    new_name = os.path.join(base_path, file.replace('.mp3.mp3', ".wav"))
    old_name = os.path.join(base_path, file)
    os.rename(old_name, new_name)
#os.rename()

In [32]:
def precalculate_and_save_one_epoch(folder, run, dataset):
    for sample in tqdm(range(len(dataset))):
        k = dataset[sample][0]
        name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)

In [33]:
def save_sample(sample, dataset, run, folder):
    try:
        k = dataset[sample][0]
        name = dataset.X[sample].split("/")[-1].split(".")[0]
        #name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)
    except Exception as exp:
        print(exp)

In [34]:
def precalculate_and_save_one_epoch_parallel(run, folder, dataset):
    before = time.time()
    with ThreadPoolExecutor(max_workers=16) as executor:
        for sample in range(len(dataset)):
            future = executor.submit(save_sample, sample, dataset, run, folder)
    
    print(time.time() - before)

In [35]:
def precalculate_and_save_one_epoch_joblib(run, folder, dataset):
    Parallel(n_jobs=32, verbose=5, backend="multiprocessing")(delayed(save_sample)(sample, dataset, run, folder)
                        for sample in range(len(dataset)))

!rm -rf ../data/files/raw_mels
!rm -rf ../data/files/raw_mels_val

In [37]:
for num in range(0, 1):
    precalculate_and_save_one_epoch_joblib(num, "../data/files/raw_mels", common_dataset)
    #precalculate_and_save_one_epoch_joblib(num, "../data/files/raw_mels_val", val_dataset)

[Parallel(n_jobs=32)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=32)]: Done 268 tasks      | elapsed:    1.8s
[Parallel(n_jobs=32)]: Done 1024 tasks      | elapsed:    4.1s
[Parallel(n_jobs=32)]: Done 1996 tasks      | elapsed:    7.1s
[Parallel(n_jobs=32)]: Done 3184 tasks      | elapsed:   10.5s
[Parallel(n_jobs=32)]: Done 4588 tasks      | elapsed:   14.7s
[Parallel(n_jobs=32)]: Done 6208 tasks      | elapsed:   19.6s
[Parallel(n_jobs=32)]: Done 8044 tasks      | elapsed:   25.0s
[Parallel(n_jobs=32)]: Done 10096 tasks      | elapsed:   31.2s
[Parallel(n_jobs=32)]: Done 12364 tasks      | elapsed:   37.9s
[Parallel(n_jobs=32)]: Done 14848 tasks      | elapsed:   45.3s
[Parallel(n_jobs=32)]: Done 17548 tasks      | elapsed:   53.3s
[Parallel(n_jobs=32)]: Done 20464 tasks      | elapsed:  1.0min
[Parallel(n_jobs=32)]: Done 23596 tasks      | elapsed:  1.2min
[Parallel(n_jobs=32)]: Don

In [41]:
len(common_dataset)

71781

In [ ]:
len(os.listdir("../data/files/raw_mels/"))